In [4]:
import pandas as pd
import numpy as np
import os, zipfile
import subprocess

In [304]:
# compare standalone models for binary classification
from numpy import mean
from numpy import std
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold, train_test_split, cross_validate

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import lightgbm as lgbm
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [6]:
import category_encoders as ce

In [7]:
dir_name = 'D:\\1mfti\ml\Home-Credit-Default-Risk'
extension = ".zip"

os.chdir(dir_name)

In [234]:
train = pd.read_csv('data/application_train.csv')
test = pd.read_csv('data/application_test.csv')

In [9]:
train['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

Дисбаланс классов

In [10]:
cat_cols = list(train.select_dtypes(include=['object']).columns)

In [119]:
num_cols = list(train.select_dtypes(include=['int64', 'float64']).columns)
num_cols.remove('TARGET')

In [239]:
x_cols = num_cols + cat_cols

In [133]:
train[num_cols]

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
4,100007,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,157500.0,254700.0,27558.0,225000.0,0.032561,-9327,-236,-8456.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307507,456252,0,72000.0,269550.0,12001.5,225000.0,0.025164,-20775,365243,-4388.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
307508,456253,0,153000.0,677664.0,29979.0,585000.0,0.005002,-14966,-7921,-6737.0,...,0,0,0,0,1.0,0.0,0.0,1.0,0.0,1.0
307509,456254,0,171000.0,370107.0,20205.0,319500.0,0.005313,-11961,-4786,-2562.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [347]:
def preprocess(train1, test1):
    y_train = train['TARGET']

    train1[num_cols] = train1[num_cols].fillna(train1[num_cols].median())
    test1[num_cols] = test1[num_cols].fillna(test1[num_cols].median())

    train1[cat_cols] = train1[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))
    test1[cat_cols] = test1[cat_cols].apply(lambda x: x.fillna(x.mode()[0]))    
    
#     train1[num_cols] = train1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
#     test1[num_cols] = test1[num_cols].apply(lambda x: pd.cut(x, bins=4, labels=False))
    TargetEncoder = ce.TargetEncoder()
    train1[cat_cols] = TargetEncoder.fit_transform(train1[cat_cols], train1['TARGET'])
    test1[cat_cols] = TargetEncoder.transform(train1[cat_cols])

    scaler = StandardScaler()
    scaler.fit(train1[x_cols])
#     train1 = scaler.fit_transform(train1)
#     test1 = scaler.fit_transform(test1)
    train1 = pd.DataFrame(scaler.transform(train1[x_cols]), index=train1.index, columns=x_cols)
    test1 = pd.DataFrame(scaler.transform(test1[x_cols]), index=test1.index, columns=x_cols)
    train1['TARGET'] = y_train

    return train1, test1

In [17]:
def get_kaggle_results():
    p = subprocess.Popen("kaggle competitions submissions  -c home-credit-default-risk", stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    print(output.decode())

In [348]:
train_norm, test_norm = preprocess(train, test)
X_train = train_norm.drop('TARGET', axis=1)
y_train = train_norm['TARGET']
X_test = test_norm

C:\Python\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [354]:
from sklearn.model_selection import RandomizedSearchCV

In [139]:
ada = RandomUnderSampler(random_state=43)
X_res, y_res = ada.fit_resample(X_train, y_train)

In [172]:
sample = ['CNT_CHILDREN',
'AMT_INCOME_TOTAL',
'DAYS_EMPLOYED',
'OWN_CAR_AGE',
'TOTALAREA_MODE',
'DAYS_LAST_PHONE_CHANGE', 'CODE_GENDER',
'FLAG_OWN_CAR', 'NAME_INCOME_TYPE', 
                      'NAME_EDUCATION_TYPE', 
                      'NAME_FAMILY_STATUS', 
                      'NAME_HOUSING_TYPE', 
                      'REGION_RATING_CLIENT', 
                      'ORGANIZATION_TYPE', ]

In [364]:
!pip install optuna

In [371]:
import optuna
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

In [369]:
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        # "device_type": trial.suggest_categorical("device_type", ['gpu']),
        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 200, 10000, step=100),
        "lambda_l1": trial.suggest_int("lambda_l1", 0, 100, step=5),
        "lambda_l2": trial.suggest_int("lambda_l2", 0, 100, step=5),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0, 15),
        "bagging_fraction": trial.suggest_float(
            "bagging_fraction", 0.2, 0.95, step=0.1
        ),
        "bagging_freq": trial.suggest_categorical("bagging_freq", [1]),
        "feature_fraction": trial.suggest_float(
            "feature_fraction", 0.2, 0.95, step=0.1
        ),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1121218)

    cv_scores = np.empty(5)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = lgbm.LGBMClassifier(objective="binary", **param_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            eval_metric="binary_logloss",
            early_stopping_rounds=100,
            callbacks=[
                LightGBMPruningCallback(trial, "binary_logloss")
            ],  # Add a pruning callback
        )
        preds = model.predict_proba(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

In [372]:
study = optuna.create_study(direction="minimize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, n_trials=20)

[I 2022-01-18 20:44:19,078] A new study created in memory with name: LGBM Classifier
C:\Python\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(


[1]	valid_0's binary_logloss: 0.280163
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.279921
[3]	valid_0's binary_logloss: 0.278673
[4]	valid_0's binary_logloss: 0.277527
[5]	valid_0's binary_logloss: 0.277069
[6]	valid_0's binary_logloss: 0.276828
[7]	valid_0's binary_logloss: 0.275744
[8]	valid_0's binary_logloss: 0.275505
[9]	valid_0's binary_logloss: 0.274515
[10]	valid_0's binary_logloss: 0.274135
[11]	valid_0's binary_logloss: 0.273892
[12]	valid_0's binary_logloss: 0.272997
[13]	valid_0's binary_logloss: 0.272817
[14]	valid_0's binary_logloss: 0.271982
[15]	valid_0's binary_logloss: 0.271708
[16]	valid_0's binary_logloss: 0.271511
[17]	valid_0's binary_logloss: 0.270767
[18]	valid_0's binary_logloss: 0.270552
[19]	valid_0's binary_logloss: 0.26982
[20]	valid_0's binary_logloss: 0.269565
[21]	valid_0's binary_logloss: 0.269404
[22]	valid_0's binary_logloss: 0.268773
[23]	valid_0's binary_logloss: 0.268599
[24]	valid_0's binary_loglos

[203]	valid_0's binary_logloss: 0.253844
[204]	valid_0's binary_logloss: 0.253827
[205]	valid_0's binary_logloss: 0.253827
[206]	valid_0's binary_logloss: 0.253808
[207]	valid_0's binary_logloss: 0.253796
[208]	valid_0's binary_logloss: 0.253788
[209]	valid_0's binary_logloss: 0.25378
[210]	valid_0's binary_logloss: 0.25377
[211]	valid_0's binary_logloss: 0.253759
[212]	valid_0's binary_logloss: 0.253748
[213]	valid_0's binary_logloss: 0.253748
[214]	valid_0's binary_logloss: 0.253737
[215]	valid_0's binary_logloss: 0.253732
[216]	valid_0's binary_logloss: 0.253723
[217]	valid_0's binary_logloss: 0.253711
[218]	valid_0's binary_logloss: 0.253711
[219]	valid_0's binary_logloss: 0.253703
[220]	valid_0's binary_logloss: 0.253693
[221]	valid_0's binary_logloss: 0.253684
[222]	valid_0's binary_logloss: 0.253684
[223]	valid_0's binary_logloss: 0.253684
[224]	valid_0's binary_logloss: 0.253684
[225]	valid_0's binary_logloss: 0.253684
[226]	valid_0's binary_logloss: 0.253666
[227]	valid_0's bi

C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 2 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 3 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 4 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 5 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_tria

[1]	valid_0's binary_logloss: 0.280169
Training until validation scores don't improve for 100 rounds
[2]	valid_0's binary_logloss: 0.279919
[3]	valid_0's binary_logloss: 0.278746
[4]	valid_0's binary_logloss: 0.277579
[5]	valid_0's binary_logloss: 0.277077
[6]	valid_0's binary_logloss: 0.276844
[7]	valid_0's binary_logloss: 0.275809
[8]	valid_0's binary_logloss: 0.275585


...

C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 124 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 125 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 126 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 127 is already reported.
  warnings.warn(



[128]	valid_0's binary_logloss: 0.253644
[129]	valid_0's binary_logloss: 0.253644
[130]	valid_0's binary_logloss: 0.253644


C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 128 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 129 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 130 is already reported.
  warnings.warn(
C:\Python\lib\site-packages\optuna\trial\_trial.py:590: UserWarning: The reported value is ignored because this `step` 131 is already reported.
  warnings.warn(


[131]	valid_0's binary_logloss: 0.253644
[132]	valid_0's binary_logloss: 0.253644
Early stopping, best iteration is:
[32]	valid_0's binary_logloss: 0.253644


[I 2022-01-18 20:51:54,042] Trial 17 finished with value: 0.25335608043545144 and parameters: {'n_estimators': 10000, 'learning_rate': 0.26465934423121595, 'num_leaves': 1500, 'max_depth': 9, 'min_data_in_leaf': 3400, 'lambda_l1': 35, 'lambda_l2': 40, 'min_gain_to_split': 7.828780000973856, 'bagging_fraction': 0.9, 'bagging_freq': 1, 'feature_fraction': 0.8}. Best is trial 2 with value: 0.25288992028965146.
C:\Python\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-01-18 20:51:58,249] Trial 18 pruned. Trial was pruned at iteration 0.
C:\Python\lib\site-packages\optuna\distributions.py:545: UserWarning: The distribution is specified by [0.2, 0.95] and q=0.1, but the range is not divisible by `q`. It will be replaced by [0.2, 0.9].
  warnings.warn(
[I 2022-01-18 20:52:02,322] Trial 19 pruned. Trial was pruned at iteration 0.

In [389]:

print(f"\tBest value (binary_logloss): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")    


	Best value (binary_logloss): 0.25289
	Best params:
		n_estimators: 10000
		learning_rate: 0.29061248921782606
		num_leaves: 1900
		max_depth: 10
		min_data_in_leaf: 6400
		lambda_l1: 15
		lambda_l2: 35
		min_gain_to_split: 2.9507726067280275
		bagging_fraction: 0.7
		bagging_freq: 1
		feature_fraction: 0.8


In [377]:
study.best_params.items()

dict_items([('n_estimators', 10000), ('learning_rate', 0.29061248921782606), ('num_leaves', 1900), ('max_depth', 10), ('min_data_in_leaf', 6400), ('lambda_l1', 15), ('lambda_l2', 35), ('min_gain_to_split', 2.9507726067280275), ('bagging_fraction', 0.7), ('bagging_freq', 1), ('feature_fraction', 0.8)])

In [381]:
# LR_model_2 = LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.1)
LR_model_2 = lgbm.LGBMClassifier(**study.best_params)
LR_model_2.fit(X_train, y_train)
predict_2 = LR_model_2.predict_proba(X_test)


In [385]:
predict_2 = 1- predict_2

In [386]:
write_submission_2 = pd.read_csv('data/sample_submission.csv')
write_submission_2['TARGET'] = predict_2
write_submission_2.to_csv('predict_under_sample.csv', index= False)

In [387]:
!kaggle competitions submit -c home-credit-default-risk -f predict_under_sample.csv -m "Message"

Successfully submitted to Home Credit Default Risk



  0%|          | 0.00/1.31M [00:00<?, ?B/s]
  1%|          | 8.00k/1.31M [00:00<00:48, 28.2kB/s]
  7%|6         | 88.0k/1.31M [00:00<00:04, 286kB/s] 
 10%|#         | 136k/1.31M [00:00<00:03, 335kB/s] 
 15%|#4        | 200k/1.31M [00:00<00:02, 429kB/s]
 21%|##        | 280k/1.31M [00:00<00:02, 509kB/s]
 30%|##9       | 400k/1.31M [00:00<00:01, 709kB/s]
 36%|###5      | 480k/1.31M [00:00<00:01, 695kB/s]
 43%|####3     | 576k/1.31M [00:01<00:01, 776kB/s]
 63%|######2   | 840k/1.31M [00:01<00:00, 1.26MB/s]
 74%|#######4  | 992k/1.31M [00:01<00:00, 1.32MB/s]
 84%|########4 | 1.10M/1.31M [00:01<00:00, 1.29MB/s]
 94%|#########3| 1.23M/1.31M [00:01<00:00, 1.27MB/s]
100%|##########| 1.31M/1.31M [00:02<00:00, 509kB/s] 


In [388]:
get_kaggle_results()

fileName                  date                 description  status    publicScore  privateScore  
------------------------  -------------------  -----------  --------  -----------  ------------  
predict_under_sample.csv  2022-01-18 16:40:11  Message      complete  0.68914      0.68275       
predict_under_sample.csv  2022-01-18 16:27:24  Message      complete  0.31085      0.31724       
predict_under_sample.csv  2022-01-18 15:14:14  Message      complete  0.33022      0.33286       
predict_under_sample.csv  2022-01-18 15:09:16  Message      complete  0.68298      0.67374       
predict_under_sample.csv  2022-01-18 15:05:44  Message      complete  0.68297      0.67374       
predict_under_sample.csv  2022-01-18 15:02:22  Message      complete  0.68297      0.67374       
predict_under_sample.csv  2022-01-18 12:04:20  Message      complete  0.70198      0.69182       
predict_under_sample.csv  2022-01-18 12:01:36  Message      complete  0.70363      0.69420       
predict_under_sample

# Отчет
Использованные модели: LogisticRegression, lgbm, SGDRegressor

Методы нормализации данных: StandardScaler, MinMaxScaler, binning

Способ обработки категориальных данных: TargetEncoder

Заполнение пропусков: медиана для числовых столбцов, мода для категориальных

Методы борьбы с дисбалансом классов: class weight = balanced, RandomUnderSampling

Лучший результат: ROC-AUC 0.71225 

LogisticRegression(C=0.0001, class_weight = 'balanced',max_iter=200, solver = 'saga',penalty = 'elasticnet', l1_ratio = 0.1), StandardScaler

